In [ ]:
import xarray as xr
from os.path import join

In [ ]:
ddir = r'/home/dirk/experiments/cama/GTSM'

In [ ]:

fn = r'all_fes_data_198_masked.nc'
dst = xr.open_mfdataset(join(ddir, fn))

In [ ]:
fn = r'global_model_surgelevel_1980.nc'
dss = xr.open_mfdataset(join(ddir, fn))
dss

In [ ]:
surge_range = dss.waterlevel.max(dim='time') - dss.waterlevel.min(dim='time')
surge_invalid = dss.stations[surge_range == 0].values
surge_invalid

In [ ]:
tide_range = dst.tide.max(dim='time') - dst.tide.min(dim='time')
tide_invalid = dst.stations[tide_range == 0].values
tide_invalid

In [ ]:
import numpy as np
invalid_stations = np.unique(np.concatenate([surge_invalid, tide_invalid]))
invalid_stations

In [ ]:
import pandas as pd
stations = dss.drop(['waterlevel', 'time']).squeeze().to_dataframe()
valid_stations = [s for s in stations.index if s not in invalid_stations]
stations.loc[valid_stations, :].to_csv(join(ddir, 'gtsm_valid.csv'))
stations.loc[invalid_stations, :].to_csv(join(ddir, 'gtsm_invalid.csv'))

In [ ]:
import pandas as pd
from os.path import join
stations = pd.read_csv(join(ddir, 'gtsm_valid.csv'), index_col=0)

In [ ]:
import geopandas as gp
from shapely.geometry import Point
from fiona.crs import from_epsg
geoms = [Point(x, y) for x, y in zip(stations.loc[:, 'station_x_coordinate'], stations.loc[:, 'station_y_coordinate'])]
gdf = gp.GeoDataFrame(stations.loc[:, ['station_id']].copy(), geometry=geoms, crs=from_epsg(4326))
gdf.to_file(join(ddir, 'gtsm_valid.geojson'), driver='GeoJSON')
gdf.head()